'telecom.csv'

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [17]:
df=pd.read_csv('telecom.csv')
#delete the rows which conains empty values
# df=df[df['TotalCharges'] != " "].copy()
# col = pd.to_numeric(df['TotalCharges'], downcast="float")
# df['TotalCharges']=col

data_x=df[df.columns.difference(['Churn','customerID','TotalCharges'])]

y=df.Churn

In [18]:
#change string columns
data_x=pd.get_dummies(data_x, columns=['DeviceProtection','InternetService','OnlineSecurity','OnlineBackup','StreamingTV','StreamingMovies','TechSupport'],drop_first=True)
# data_x.drop(columns=['DeviceProtection_No internet service','OnlineSecurity_No internet service','OnlineBackup_No internet service','StreamingTV_No internet service','StreamingMovies_No internet service','TechSupport_No internet service'], inplace=True)
data_x=pd.get_dummies(data_x, columns=['gender','Dependents','PaperlessBilling','Partner','PhoneService','MultipleLines'],drop_first=True)
# data_x.drop(columns=['MultipleLines_No phone service'],inplace=True)
data_x=pd.get_dummies(data_x, columns=['Contract','PaymentMethod'])
y=pd.get_dummies(y, drop_first=True)

Split data into 2 parts, take the small one as the final test. You will be using for each model's final evaluation

In [19]:
x_cur, x_sep, y_cur, y_sep=train_test_split(data_x, y, test_size=0.2, random_state=1)


In [6]:
len(x_cur), len(x_sep), len(y_cur), len(y_sep)

(4800, 1200, 4800, 1200)

# Build 5 classification models

logistic regression, decision tree, random forest, knn, svm

#### tips
<br>
1. use cross validation
<br>
2. use same splits for all classifiers (to have correct comparison)
<br>
3. build a table to show statistics for all classifiers (at least - accuracy, precision and recall. Any more component would be a +. For train - at least accuracy)

In [20]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import statistics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from imblearn.over_sampling import SMOTE


In [30]:
n='0'

metrics_list=[]

metrics_for=['logreg','dectree','randforest','knn','svm']

types_for=['train','test', 'sep_train','sep_test' ]

logreg=LogisticRegression()
clf = DecisionTreeClassifier(max_depth=5)
rclf = RandomForestClassifier(n_estimators=200,max_depth=5)
neigh = KNeighborsClassifier(n_neighbors=3)
svmclf = svm.SVC(C=500,kernel='linear')

sm = SMOTE()

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(x_cur):
    x_train, x_test = x_cur.iloc[train_index], x_cur.iloc[test_index]
    y_train, y_test = y_cur.iloc[train_index], y_cur.iloc[test_index]
    
    for tp in metrics_for:
        print(tp)
        
        pred_train=0
        pred_test=0

        if (tp=='logreg'):
            
            
            
            logreg.fit(x_train,y_train)
    
            pred_train=logreg.predict(x_train)
            pred_test=logreg.predict(x_test)
            
            n='1'
            
        elif (tp=='dectree'):
   
            
                                                   
            clf = clf.fit(x_train,y_train)

            pred_train = clf.predict(x_train)
            pred_test = clf.predict(x_test)
            
            n='2'  
        elif(tp=='randforest'):
                                                   
         
            rclf.fit(x_train,y_train)
            pred_train = rclf.predict(x_train)
            pred_test = rclf.predict(x_test)
            
            n='3' 
        elif(tp=='knn'):
            
            neigh.fit(x_train,y_train)
            pred_train = neigh.predict(x_train)
            pred_test = neigh.predict(x_test)
            
            n='4'
        elif(tp=='svm'):
            
            
            svmclf.fit(x_train,y_train)
            
            pred_train = svmclf.predict(x_train)
            pred_test = svmclf.predict(x_test)
            
            n='5'
            
        metrics_list.append([n+'11',tp+'_train_acc',metrics.accuracy_score(y_train, pred_train)])
        metrics_list.append ([n+'21',tp+'_test_acc',metrics.accuracy_score(y_test, pred_test)])

        metrics_list.append([n+'12',tp+'_train_perc',metrics.precision_score(y_train, pred_train, average=None)[0]])
        metrics_list.append([n+'22',tp+'_test_perc',metrics.precision_score(y_test, pred_test, average=None)[0]])
        
        metrics_list.append([n+'13',tp+'_train_recall',metrics.recall_score(y_train, pred_train)])
        metrics_list.append([n+'23',tp+'_test_recall',metrics.recall_score(y_test, pred_test)])
  


#create final model and test on sep datas

x_train=x_cur
y_train=y_cur
x_test=x_sep
y_test=y_sep

# x_train, y_train = sm.fit_sample(x_cur, y_cur)

for tp in metrics_for:

    pred_train=0
    pred_test=0

    if (tp=='logreg'):

        logreg.fit(x_train,y_train)

        pred_train=logreg.predict(x_train)
        pred_test=logreg.predict(x_test)

        n='1'

    elif (tp=='dectree'):

        clf = clf.fit(x_train,y_train)

        pred_train = clf.predict(x_train)
        pred_test = clf.predict(x_test)

        n='2'  
    elif(tp=='randforest'):
        rclf.fit(x_train,y_train)
        pred_train = rclf.predict(x_train)
        pred_test = rclf.predict(x_test)
        
        n='3' 
    elif(tp=='knn'):
        neigh.fit(x_train,y_train)
        pred_train = neigh.predict(x_train)
        pred_test = neigh.predict(x_test)

        n='4'
    elif(tp=='svm'):

        svmclf.fit(x_train,y_train)
            
        pred_train = svmclf.predict(x_train)
        pred_test = svmclf.predict(x_test)

        n='5'

    metrics_list.append([n+'31',tp+'_sep_train_acc',metrics.accuracy_score(y_train, pred_train)])
    metrics_list.append ([n+'41',tp+'_sep_test_acc',metrics.accuracy_score(y_test, pred_test)])

    metrics_list.append([n+'32',tp+'_sep_train_perc',metrics.precision_score(y_train, pred_train, average=None)[0]])
    metrics_list.append([n+'42',tp+'_sep_test_perc',metrics.precision_score(y_test, pred_test, average=None)[0]])

    metrics_list.append([n+'33',tp+'_sep_train_recall',metrics.recall_score(y_train, pred_train)])
    metrics_list.append([n+'43',tp+'_sep_test_recall',metrics.recall_score(y_test, pred_test)])
            

metrics_df= pd.DataFrame(metrics_list)
metrics_dfg=metrics_df.groupby([0,1]).mean()


    
statistica=pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])

for row in metrics_dfg.iterrows():
    metr=int(row[0][0][0])-1
    tp=int(row[0][0][1])-1
    col_index=int(row[0][0][2])-1
    
    metr_v=metrics_for[metr]
    tp_v=types_for[tp]
   
    col=statistica.columns[col_index]
    
    
  
    value=row[1].values

    
    row=metr_v+'_'+tp_v
  
  
    
    if(row not in statistica.index):    
        statistica.loc[row]=[0,0,0]
  

    statistica[col][row]=value[0]
   
   




 
    

logreg
dectree
randforest
knn
svm
logreg
dectree
randforest
knn
svm
logreg
dectree
randforest
knn
svm
logreg
dectree
randforest
knn
svm
logreg
dectree
randforest
knn
svm


In [31]:
statistica

,Accuracy,Precision,Recall
logreg_train,0.809479,0.850455,0.574126
logreg_test,0.807292,0.84848,0.567795
logreg_sep_train,0.811042,0.851571,0.577487
logreg_sep_test,0.804167,0.854754,0.513986
dectree_train,0.804896,0.858393,0.607636
dectree_test,0.792292,0.850331,0.585713
dectree_sep_train,0.800833,0.870742,0.657672
dectree_sep_test,0.805,0.886364,0.65035
randforest_train,0.805885,0.824849,0.465403
randforest_test,0.796042,0.818142,0.443803


In [22]:
statistica

,Accuracy,Precision,Recall
logreg_train,0.810093,0.850523,0.573938
logreg_test,0.805625,0.847597,0.566329
logreg_sep_train,0.811042,0.851571,0.577487
logreg_sep_test,0.804167,0.854754,0.513986
dectree_train,0.803171,0.858806,0.610475
dectree_test,0.790625,0.850694,0.590532
dectree_sep_train,0.800833,0.870742,0.657672
dectree_sep_test,0.804167,0.885358,0.646853
randforest_train,0.803148,0.82208,0.455069
randforest_test,0.798125,0.818983,0.446018


In [9]:
#keep
statistica 

,Accuracy,Precision,Recall
logreg_train,0.838835,0.838978,0.838994
logreg_test,0.78625,0.864443,0.644399
logreg_sep_train,0.835427,0.838032,0.839281
logreg_sep_test,0.783333,0.888361,0.671329
dectree_train,0.817455,0.834793,0.841985
dectree_test,0.764583,0.880605,0.711477
dectree_sep_train,0.818727,0.856823,0.87211
dectree_sep_test,0.743333,0.896597,0.723776
randforest_train,0.808542,0.847323,0.864208
randforest_test,0.756042,0.901417,0.780141


In [9]:
statistica

,Accuracy,Precision,Recall
logreg_train,0.810093,0.850523,0.573938
logreg_test,0.805625,0.847597,0.566329
logreg_sep_train,0.811042,0.851571,0.577487
logreg_sep_test,0.804167,0.854754,0.513986
dectree_train,0.803171,0.858806,0.610475
dectree_test,0.790625,0.850694,0.590532
dectree_sep_train,0.800833,0.870742,0.657672
dectree_sep_test,0.805,0.886364,0.65035
randforest_train,0.804398,0.823898,0.462467
randforest_test,0.796875,0.819325,0.448346


#### choose the best model. Either retrain it or copy it in the cell below

In [32]:
# final_model 
from sklearn.metrics import classification_report
clf.fit(x_cur,y_cur)
pred=clf.predict(x_sep)
print(classification_report(y_sep, pred))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87       914
           1       0.58      0.65      0.61       286

    accuracy                           0.81      1200
   macro avg       0.73      0.75      0.74      1200
weighted avg       0.81      0.81      0.81      1200



In [34]:
from sklearn.metrics import classification_report
clf.fit(x_cur,y_cur)
pred=clf.predict(x_sep)
print(classification_report(y_sep, pred))
print(confusion_matrix(y_sep, pred))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87       914
           1       0.58      0.65      0.61       286

    accuracy                           0.81      1200
   macro avg       0.73      0.75      0.74      1200
weighted avg       0.81      0.81      0.81      1200

[[781 133]
 [101 185]]


In [ ]:
              precision    recall  f1-score   support

           0       0.91      0.76      0.83       914
           1       0.50      0.77      0.61       286

    accuracy                           0.76      1200
   macro avg       0.71      0.77      0.72      1200
weighted avg       0.82      0.76      0.78      1200